In [2]:
import ee
ee.Authenticate()
ee.Initialize()


Successfully saved authorization token.


In [46]:
bands = [
    'sm_surface', 
    'sm_surface_wetness',
    'soil_temp_layer1',
    'land_evapotranspiration_flux',
    'soil_water_infiltration_flux',
    'precipitation_total_surface_flux',
    'vegetation_greenness_fraction', 
    'leaf_area_index',
    'land_fraction_saturated']

bounds = ee.Geometry.BBox(-20,-40,60,60)

In [47]:
sm = (ee.ImageCollection("NASA/SMAP/SPL4SMGP/007")
    .filterBounds(bounds)
    .select(bands)
    #.select('sm_surface')
    )

In [48]:
start = ee.Date('2015-04-01')
end = ee.Date('2023-01-01')
n_months = end.difference(start, 'month').subtract(1)
months = ee.List.sequence(0, n_months).map(lambda n : start.advance(n, 'month'))

In [49]:
def makeMonthlyComposite(date):
  date = ee.Date(date)
  
  return (sm
          .filterDate(date, date.advance(1, 'month'))
          .mean()
          .set("system:index", date.format("YYYY-MM")))

In [50]:
monthly_sm = ee.ImageCollection(months.map(makeMonthlyComposite))

In [51]:
#monthly_sm.aggregate_array("system:index").getInfo()

In [54]:
for b in ['soil_water_infiltration_flux', 'sm_surface_wetness']: # bands:
    
    image = monthly_sm.select(b).toBands()
    
    task = ee.batch.Export.image.toDrive(
        image=image,
        description=f'{b}',
        folder = 'smap_africa',
        region = bounds,
        dimensions = "1920x2400",
        #scale = 5000,
        #crs = 'EPSG:25832'
    )
    task.start()

In [53]:
task.status()

{'state': 'READY',
 'description': 'land_fraction_saturated',
 'creation_timestamp_ms': 1678699832502,
 'update_timestamp_ms': 1678699832502,
 'start_timestamp_ms': 0,
 'task_type': 'EXPORT_IMAGE',
 'id': 'GH5PEWUHJSIOHG4LYQ7GJ26H',
 'name': 'projects/earthengine-legacy/operations/GH5PEWUHJSIOHG4LYQ7GJ26H'}